In [27]:
import random
import datetime
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch, mm
from google.colab import drive

def generate_random_items(num_items):
  """Generates a list of random stationery items with quantities and prices."""
  stationery_items = ["Pen", "Pencil", "Notebook", "Eraser", "Ruler", "Sharpener", "Marker", "Glue", "Stapler", "Paper Clips"]
  price_ranges = {"Pen": (10, 30), "Pencil": (5, 15), "Notebook": (50, 100), "Eraser": (10, 20),
                  "Ruler": (20, 40), "Sharpener": (15, 30), "Marker": (25, 50), "Glue": (30, 60),
                  "Stapler": (100, 150), "Paper Clips": (5, 10)}

  items = []
  for _ in range(num_items):
    item = random.choice(stationery_items)
    quantity = random.randint(1, 5)
    price = random.randint(price_ranges[item][0], price_ranges[item][1])
    items.append({"name": item, "quantity": quantity, "price": price})
  return items

def create_receipt(data):
  """
  Creates a payment receipt as a PDF file and saves it to Google Drive.

  Args:
    data (dict): A dictionary containing receipt data.
  """

  drive.mount('/content/drive')

  filename = '/content/drive/MyDrive/CipherByte Tasks/Receipts/receipt_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '.pdf'
  doc = canvas.Canvas(filename, pagesize=letter)

  page_width, page_height = letter
  border_x, border_y = 50, 50
  border_width, border_height = page_width - 100, page_height - 100

  # Draw the border box
  p = doc.beginPath()
  p.moveTo(border_x, border_y)
  p.lineTo(border_x + border_width, border_y)
  p.lineTo(border_x + border_width, border_y + border_height)
  p.lineTo(border_x, border_y + border_height)
  p.close()
  doc.rect(border_x, border_y, border_width, border_height)

  # Receipt header
  doc.setFont("Helvetica-Bold", 16)
  doc.drawCentredString(border_x + border_width / 2, border_y + border_height - 30, "Payment Receipt")

  # Store information
  try:
    doc.setFont("Helvetica", 12)
    doc.drawString(border_x + 20, border_y + border_height - 60, f"Store Name: {data['store_name']}")
    doc.drawString(border_x + 20, border_y + border_height - 80, f"Store Address: {data['store_address']}")
  except KeyError as e:
    print(f"Error: Missing key in store information: {e}")

  # Transaction details
  doc.setFont("Helvetica-Bold", 14)
  doc.drawCentredString(border_x + border_width / 2, border_y + border_height - 120, "Transaction Details")

  doc.setFont("Courier", 12)  # Use fixed-width font for better alignment
  doc.drawString(border_x + 20, border_y + border_height - 150, f"Transaction ID: {data['transaction_id']}")
  doc.drawString(border_x + 20, border_y + border_height - 170, f"Date: {data['date']}")
  doc.drawString(border_x + 20, border_y + border_height - 190, f"Total Amount: Rs. {data['total_amount']:.2f}")

  # Itemized list
  doc.setFont("Helvetica-Bold", 12)
  doc.drawCentredString(border_x + border_width / 2, border_y + border_height - 220, "Items Purchased")

  doc.setFont("Courier", 10)  # Use fixed-width font for better alignment
  y = border_y + border_height - 250
  for item in data['items']:
    price_str = f"Rs. {item['price']:.2f}"
    doc.drawString(border_x + 20, y, f"{item['quantity']} x {item['name']} - {price_str}")
    y -= 20

  # Payment method
  doc.setFont("Helvetica-Bold", 12)
  doc.drawCentredString(border_x + border_width / 2, border_y + 100, "Payment Method")
  doc.setFont("Helvetica", 12)
  doc.drawCentredString(border_x + border_width / 2, border_y + 80, f"{data['payment_method']}")

  # Footer
  doc.setFont("Helvetica", 8)
  doc.drawCentredString(border_x + border_width / 2, border_y + 20, "This receipt is for your records only.")

  doc.save()

if __name__ == "__main__":
  receipt_data = {
    "store_name": "CipherByte Store",
    "store_address": "Kolkata, West Bengal",
    "transaction_id": "12345",
    "date": "2023-11-22",
    "total_amount": 0,  # This will be calculated later
    "items": [],  # Will be populated by generate_random_items
    "payment_method": "Cash"
  }

  receipt_data['items'] = generate_random_items(random.randint(3, 5))
  receipt_data['total_amount'] = sum(item['quantity'] * item['price'] for item in receipt_data['items'])

  create_receipt(receipt_data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
